Stratif_essai

In [1]:
%load_ext memory_profiler
%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
#from glob import glob
from pathlib import Path
import os

import numpy as np

import scipy.interpolate as itp
import xarray as xr
from xorca.lib import load_xorca_dataset
#import gsw

print("ca marche")

ca marche


In [2]:
from dask.distributed import Client, LocalCluster, wait
# Start a local cluster with 4 single-threaded (HDF is triggering errors
# if they are multi-threaded) workers which get only 1500 MiB of memory each.
cluster = LocalCluster(n_workers=8, threads_per_worker=1, memory_limit=1500e6,
                       silence_logs=50)
client = Client(address=cluster)
client

Client Scheduler: tcp://127.0.0.1:53113 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 11.18 GiB


In [3]:
data_path = Path("/Volumes/Calcul/eNATL60")

data_fname = "eNATL60NOE-BLBT02_y2010m01d??.1h_*_0-botm.nc"
grid_fname = ["mesh_*_eNATL60NOE_3.6.nc", "mask_eNATL60NOE_3.6.nc"]

data_files = list(data_path.glob(data_fname))
grid_files = sum([list(data_path.glob(fna)) for fna in grid_fname],[]) # trick to flatten: sum([list_of_list],[])

In [4]:
%%time
%%memit
ds_tot = load_xorca_dataset(data_files=data_files, aux_files=grid_files,
                              decode_cf=True, model_config="nest"
                             )
print("dataset is {:.1f} GB".format(ds_tot.nbytes/1e9))

dataset is 75.3 GB
peak memory: 600.67 MiB, increment: 443.42 MiB
CPU times: user 25.8 s, sys: 1.25 s, total: 27.1 s
Wall time: 34.2 s


In [ ]:
ds_tot

In [ ]:
%%time
%%memit
### compute mean and store in a netCDF file

do_cal = False
mean_file = "eNATL60NOE_mean_strat.nc"
mean_file = data_path/mean_file

if do_cal:
    ds_moy = ds_tot.get(["vosaline","votemper"]).mean("t").assign_coords(z_l=ds_tot.z_l)
    ds_moy.to_netcdf(mean_file, mode="w")
    del ds_moy
    
ds_moy = xr.load_dataset(mean_file)

if "z_l" not in ds_moy: # for previous versions
    ds_moy = ds_moy.assign_coords(z_l=ds_tot.z_l)
    
print("ds_moy is {:.2e} MB".format(ds_moy.nbytes/1e6))

In [ ]:
ds_moy = ds_moy.assign_coords(e3w=ds_tot.e3w)

In [ ]:
ds_moy

In [ ]:
from xgcm import Grid

#metrics = {
#    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
#    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
#    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w'], # Z distances
#    #('X', 'Y'): [] # Areas TODO
#}
#grid = Grid(ds_tot, metrics=metrics, periodic=False)
#grid

In [ ]:
_grav = 9.81
_zmet = "e3w"
_temp = "votemper"
_salt = "vosaline"
_zdim = "Z"
_zcoord = "depth_l"

In [ ]:
def eosbn2(ds, grid=None, z_reverse=True):
    """ eosbn2 (from CDFTOOLS function eosbn2 in eos.f90)
    if z_reverse: assume that depth is negative and z-grid oriented downwards while weights are positive
    """
    temp = _temp
    salt = _salt
    zdim = _zdim
    zmetric = _zmet
    grav = _grav
    zcoord = _zcoord
    
    if grid is None: # no xgcm grid passed: use one without metrics (assume mid-depth points = w points)
        grid = Grid(ds)
    
    z_sign = -1. if z_reverse else 1.
    
    zt = grid.interp(ds[temp], zdim, boundary="extend")
    zs = grid.interp(ds[salt], zdim, boundary="extend") - 35.0
    zh = z_sign * ds[zcoord] # take minus (i.e. positive depth)
    
    zalbet = ( ( ( - 0.255019e-07 * zt + 0.298357e-05 ) * zt     #   ! ratio alpha/beta
                                       - 0.203814e-03 ) * zt   
                                       + 0.170907e-01 ) * zt \
           + 0.665157e-01                                    \
           +     ( - 0.678662e-05 * zs                         
                   - 0.846960e-04 * zt + 0.378110e-02 ) * zs \
           +   ( ( - 0.302285e-13 * zh                         
                   - 0.251520e-11 * zs                         
                   + 0.512857e-12 * zt * zt           ) * zh   
                   - 0.164759e-06 * zs                         
                +(   0.791325e-08 * zt - 0.933746e-06 ) * zt   
                                       + 0.380374e-04 ) * zh

    zbeta  = ( ( -0.415613e-09 * zt + 0.555579e-07 ) * zt        #   ! beta
                                    - 0.301985e-05 ) * zt     \
           + 0.785567e-03                                     \
           + (     0.515032e-08 * zs                           
                 + 0.788212e-08 * zt - 0.356603e-06 ) * zs    \
           +(  (   0.121551e-17 * zh                           
                 - 0.602281e-15 * zs                           
                 - 0.175379e-14 * zt + 0.176621e-12 ) * zh     
                                     + 0.408195e-10   * zs     
             + ( - 0.213127e-11 * zt + 0.192867e-09 ) * zt     
                                     - 0.121555e-07 ) * zh
        
    if "_metrics" in dir(grid):
        try:
            eosbn2 = z_sign * grav * zbeta * ( zalbet * grid.derivative(ds[temp], zdim, boundary="extend") 
                             - grid.derivative(ds[salt], zdim, boundary="extend") )
        except:
            if zmetric not in ds:
                raise ValueError("no metric found in dataset")
            eosbn2 = z_sign * grav * zbeta * ( zalbet * grid.diff(ds[temp], zdim, boundary="extend") 
                             - grid.diff(ds[salt], zdim, boundary="extend") ) / ds[zmetric]
    else:
        eosbn2 = z_sign * grav * zbeta * ( zalbet * grid.diff(ds[temp], zdim, boundary="extend") 
                             - grid.diff(ds[salt], zdim, boundary="extend") ) / ds[zmetric]

    return eosbn2

In [ ]:
%%time
ds_moy["Nsqr"] = eosbn2(ds_moy.where(ds_moy.tmask), grid=None).astype(np.float32, casting="same_kind")
ds_moy.Nsqr.isel(x_c=1, y_c=1).plot(y="depth_l")


In [ ]:
%%time
ds_moy.to_netcdf(mean_file, mode="w")